# Chapter 2

In this chapter, we'll cover the following recipes:
- Preparing a task
- Setting up a cron job
- Capturing errors and problems
- Sending email notifications

We'll begin by going over how we ought to prepare a task before we automate it.

### Preparing a task

It all starts with defining precisely the woek that needs to be executed, and designing it in a way that deosn't require human intervention to run.

Some ideal characteristic points are as follows:

1. **Sinble, clear entry point:** No confusion on how to start the task.
2. **Clear parameters:** If there are any parameters, they should be as explicit as possible.
3. **No interactivity:** Stopping the execution to request information from the user is not possible.
4. **The result should be stored:** In order to be checked at a different time than when it runs.
5. **Clear result:** When we oversee the execution of a program ourselves, we can accept more verbose results, such as unlabeled data or extra debugging information. However, for an automated task, the final result should be as concise and to the point as possible.
6. **Errors should be logged:** To analyze what went wrong.

A command-line programs has a lot of those characteristics already. It always has a clear entry point, with defined parameters, and the result can be stored, even if just in text format. And it can be improves ensuring a config file that clarifies the parameters, and an output file.

Not that point 6 is the objective of the *Capturing errors and problems* recipe, and will be covered there. 

To avoid interactivity, do not use any function that waits for user input, such as *input*. Remeber to delete debugger breakpoints!.


### Getting ready

We'll start by following a structure in which a main function will serve as the entry point, and all parameters are supplied to it.

This is the same basic structure that was presented in the *Adding command-line arguments* recipe in *Chapter 1, Let's Begin Out Automation Journey.*

### How to do it...
1. Prepare the following command-line  program by multiplying two numbers, and save it as `prepare_task_step1.py`

In [ ]:
import argparse

def main(number, other_number):
    result = number * other_number
    print(f'The result is {result}')
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-n1', type=int, help= 'A number', default=1)
    parser.add_argument('-n2', type=int, help='Another number', default=1)
    
    args=parser.parse_args()
    main(args.n1, args.n2)

2. Update the file to define a config file that contains both arguments, and save it as `prepare_task_step3.py`. Note that defining a config file overwrites any command-line parameters:

In [ ]:
import argparse
import configparser

def main(number, other_number):
    result = number * other_number
    print(f'The result is {result}')
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-n1', type=int, help='A number', default=1)
    parser.add_argument('-n2', type=int, help='Another number', default=1)
    parser.add_argument('--config', '-c', type=argparse.FileType('r'), help='config file')
    args = parser.parse_args()
    
    if args.config:
        config = configparser.ConfigParser()
        config.read_file(args.config)
        # Transforming values into integers
        args.n1 = int(config['ARGUMENTS']['n1'])
        args.n2 = int(config['ARGUMENTS']['n2'])
    
    main(args.n1, args.n2)

3. Create the config file, config.ini. See the ARGUMENTS section and the `n1` and `n2` values:

4. Run teh command with the config file. Note that the config file overwrites the command-line parameters, as described in *step 2*:

`$python3 prepare_task_step3.py -c config.ini`

`The result is 35`

`$ python3 prepare_task_step3.py -c config.ini -n1 2 -n2 3`

`The resul is 35`

5. Add a parameter to store the result in a file, and save it as `prepare_task_step6.py`

In [ ]:
import argparse
import sys
import configparser

def main(number, other_number, output):
    result = number * other_number
    print(f'The result is {result}', file=output)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-n1', type=int, help='A number', default = 1)
    parser.add_argument('-n2', type=int, help='Another number', default=1)
    parser.add_argument('--config', '-c', type=argparse.FileType('r'), help='config file')
    parser.add_argument('-o', dest='output', type=argparse.FileType('w'), help='output file', default=sys.stdout)

    args = parser.parse_args()

    if args.config:
        config = configparser.ConfigParser()
        config.read_file(args.config)

        # Transforming values into integers
        args.n1 = int(config["ARGUMENTS"]["n1"])
        args.n2 = int(config["ARGUMENTS"]["n2"])
        
    main(args.n1, args.n2, args.output)

6. Run the result to check that it's sending the output to the defined file. Note that there's no output outside the result files:

```system
$ python3 prepare_task_step6.py -n1 3 -n2 5 -o result.txt
$ cat result.txt
The result is 15
$ python prepare_task_step6.py -c config.ini -0 result2.txt
$ cat result2.txt
The result is 35
```
